# Get config jsons from fine-tuning experiments and analyze

## Ny kjøring i 2024


In [9]:
import json, os, sys, io, shutil
from pathlib import Path
import pandas as pd
from datetime import date
import zipfile


In [2]:
# scp -r ec-egilron@fox.educloud.no:/fp/homes01/u01/ec-egilron/sqlabel-github/0816_all-logs.zip tsa-0816
# 08231629_configs.zip

In [20]:
configs_path = Path("/home/egil/gits_wsl/seq-label-github/configs/fox")

name_filter = "01150927"
jsons = [f for f in configs_path.iterdir() if f.name.startswith(name_filter) and  f.name.endswith("json")]
len(jsons)


42

In [21]:
def copyover(file_l:list[Path], dest_folder:str):
    dest_folder = Path("history", dest_folder)
    dest_folder.mkdir(exist_ok=True, parents=True)
    for f in file_l:
        shutil.copy(f, dest_folder)


In [22]:

records = []
root_keys = ["timestamp",  "task",  "machinery",  "best_epoch" ]
args_keys = [ 'model_name_or_path', 'task_name', 'dataset_name', 'output_dir', 'seed' , 'per_device_train_batch_size',  'learning_rate' , 'num_train_epochs']
epoch_keys = ['eval_loss', 'eval_precision', 'eval_recall', 'eval_f1', 'eval_accuracy','epoch', 'step' ]
for j in jsons:
    jdata = json.loads(j.read_text())
    for epoch_data in jdata["train_epochs_val"]:
        record = {k:v for k,v in jdata.items() if k in root_keys}
        record["config-file"] = j.name
        record.update({k:v for k,v in jdata["args_dict"].items() if k in args_keys})
        record.update({k:v for k,v in epoch_data.items() if k in epoch_keys})
        records.append(record)
df = pd.DataFrame.from_records(records)
print("Shape:", df.shape)

xc_path = Path("excels", name_filter+"_full-report.xlsx" )

df.to_excel(xc_path, index=False)
copyover(jsons, name_filter+"_"+configs_path.stem)
df.sort_values("eval_f1", ascending=False) #.to_excel("tsa-0823/0823-finetuning_tsa.xlsx")


Shape: (840, 20)


,timestamp,task,machinery,best_epoch,config-file,model_name_or_path,dataset_name,seed,per_device_train_batch_size,task_name,output_dir,num_train_epochs,learning_rate,eval_loss,eval_precision,eval_recall,eval_f1,eval_accuracy,epoch,step
160,01150927,tsa-bin,fox,5,01150927_tsa-bin_NB-BERT_large_05.json,NbAiLab/nb-bert-large,data/tsa_binary,101,64,01150927_tsa-bin_NB-BERT_large,/cluster/work/projects/ec30/egilron/tsa-hf/011...,20,0.00005,0.208058,0.529924,0.555302,0.542316,0.956226,5.0,675
161,01150927,tsa-bin,fox,5,01150927_tsa-bin_NB-BERT_large_05.json,NbAiLab/nb-bert-large,data/tsa_binary,101,64,01150927_tsa-bin_NB-BERT_large,/cluster/work/projects/ec30/egilron/tsa-hf/011...,20,0.00005,0.340618,0.530120,0.551881,0.540782,0.957037,17.0,2295
760,01150927,tsa-bin,fox,4,01150927_tsa-bin_NB-BERT_large_12.json,NbAiLab/nb-bert-large,data/tsa_binary,101,64,01150927_tsa-bin_NB-BERT_large,/cluster/work/projects/ec30/egilron/tsa-hf/011...,20,0.00001,0.147061,0.524116,0.557583,0.540331,0.954296,4.0,540
162,01150927,tsa-bin,fox,5,01150927_tsa-bin_NB-BERT_large_05.json,NbAiLab/nb-bert-large,data/tsa_binary,101,64,01150927_tsa-bin_NB-BERT_large,/cluster/work/projects/ec30/egilron/tsa-hf/011...,20,0.00005,0.261772,0.530837,0.549601,0.540056,0.956111,9.0,1215
761,01150927,tsa-bin,fox,4,01150927_tsa-bin_NB-BERT_large_12.json,NbAiLab/nb-bert-large,data/tsa_binary,101,64,01150927_tsa-bin_NB-BERT_large,/cluster/work/projects/ec30/egilron/tsa-hf/011...,20,0.00001,0.257551,0.513374,0.568985,0.539751,0.954219,14.0,1890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,01150927,tsa-bin,fox,1,01150927_tsa-bin_NorBERT_3_large_06.json,ltg/norbert3-large,data/tsa_binary,101,64,01150927_tsa-bin_NorBERT_3_large,/cluster/work/projects/ec30/egilron/tsa-hf/011...,20,0.00005,NaN,0.000000,0.000000,0.000000,0.934185,20.0,2700
378,01150927,tsa-bin,fox,1,01150927_tsa-bin_NorBERT_3_large_06.json,ltg/norbert3-large,data/tsa_binary,101,64,01150927_tsa-bin_NorBERT_3_large,/cluster/work/projects/ec30/egilron/tsa-hf/011...,20,0.00005,NaN,0.000000,0.000000,0.000000,0.934185,19.0,2565
377,01150927,tsa-bin,fox,1,01150927_tsa-bin_NorBERT_3_large_06.json,ltg/norbert3-large,data/tsa_binary,101,64,01150927_tsa-bin_NorBERT_3_large,/cluster/work/projects/ec30/egilron/tsa-hf/011...,20,0.00005,NaN,0.000000,0.000000,0.000000,0.934185,18.0,2430
376,01150927,tsa-bin,fox,1,01150927_tsa-bin_NorBERT_3_large_06.json,ltg/norbert3-large,data/tsa_binary,101,64,01150927_tsa-bin_NorBERT_3_large,/cluster/work/projects/ec30/egilron/tsa-hf/011...,20,0.00005,NaN,0.000000,0.000000,0.000000,0.934185,17.0,2295


Report hyperparameters
cols = ["timestamp",   "machinery", 'model_name_or_path', 'task_name', 'dataset_name', 'seed' , 'per_device_train_batch_size',  'learning_rate' , 'num_train_epochs']
records = []
for col in cols:
    record = {col: df[col].unique()}

In [23]:
cols = ["timestamp",   "machinery", 'model_name_or_path','dataset_name', 'seed' , 'per_device_train_batch_size',  'learning_rate' , 'num_train_epochs']
records = {}
longest = 0
for col in cols:
    records[col]= df[col].unique()
    longest = max(longest, len(records[col]))
df_vars = pd.DataFrame(columns=cols)
for i in range(longest):
    for col in cols:
        try:
            df_vars.at[i, col] =records[col][i]
            if col=="learning_rate":
                df_vars.at[i, col] =f"{records[col][i]}"

        except:
            df_vars.at[i, col] =""
df_vars


,timestamp,machinery,model_name_or_path,dataset_name,seed,per_device_train_batch_size,learning_rate,num_train_epochs
0,01150927,fox,NbAiLab/nb-bert-large,data/tsa_intensity,101,64,1e-06,20
1,,,ltg/norbert3-small,data/tsa_binary,,,1e-05,
2,,,NbAiLab/nb-bert-base,,,,5e-05,
3,,,ltg/norbert3-base,,,,,
4,,,xlm-roberta-base,,,,,
5,,,ltg/norbert3-xs,,,,,
6,,,ltg/norbert3-large,,,,,


In [24]:
# Best epochs only
df_bests = df.loc[ df["epoch"] ==df["best_epoch"]].copy()
df_bests["learning_rate"] = df_bests["learning_rate"].apply(lambda x: f"{x:.0e}")
report = df_bests[['timestamp', 'task', 'machinery', 'best_epoch', 
       'model_name_or_path',
       'per_device_train_batch_size',  
      'learning_rate', 'eval_loss', 'eval_precision',
       'eval_recall', 'eval_f1', 'eval_accuracy',  ]].sort_values("eval_f1", ascending=False)
xc_path = Path("excels", name_filter+"_best-report.xlsx" )
report.to_excel(xc_path, index=False)
report

,timestamp,task,machinery,best_epoch,model_name_or_path,per_device_train_batch_size,learning_rate,eval_loss,eval_precision,eval_recall,eval_f1,eval_accuracy
160,01150927,tsa-bin,fox,5,NbAiLab/nb-bert-large,64,5e-05,2.080582e-01,0.529924,0.555302,0.542316,0.956226
760,01150927,tsa-bin,fox,4,NbAiLab/nb-bert-large,64,1e-05,1.470606e-01,0.524116,0.557583,0.540331,0.954296
380,01150927,tsa-bin,fox,4,ltg/norbert3-base,64,5e-05,1.945513e-01,0.500516,0.553022,0.525460,0.949819
800,01150927,tsa-bin,fox,13,ltg/norbert3-base,64,1e-05,1.992019e-01,0.518477,0.527936,0.523164,0.951285
580,01150927,tsa-bin,fox,3,NbAiLab/nb-bert-base,64,5e-05,1.733074e-01,0.543257,0.486887,0.513530,0.951942
540,01150927,tsa-bin,fox,17,NbAiLab/nb-bert-base,64,1e-05,2.551979e-01,0.487342,0.526796,0.506301,0.947464
200,01150927,tsa-bin,fox,20,xlm-roberta-base,64,5e-05,3.606558e-01,0.480999,0.505131,0.492770,0.947001
140,01150927,tsa-bin,fox,16,xlm-roberta-base,64,1e-05,2.170476e-01,0.456500,0.532497,0.491579,0.943720
700,01150927,tsa-bin,fox,13,ltg/norbert3-small,64,5e-05,2.592025e-01,0.457953,0.515393,0.484979,0.945920
220,01150927,tsa-bin,fox,20,NbAiLab/nb-bert-large,64,1e-06,1.526079e-01,0.445585,0.494869,0.468936,0.948506


In [7]:
a = 0.001
f"{a:.1e}"
df_bests

,timestamp,task,machinery,best_epoch,config-file,model_name_or_path,dataset_name,seed,per_device_train_batch_size,task_name,output_dir,num_train_epochs,learning_rate,eval_loss,eval_precision,eval_recall,eval_f1,eval_accuracy,epoch,step
0,01121628,tsa-bin,lumi,1,01121628_tsa-bin_NorBERT_3_large_05.json,ltg/norbert3-large,data/tsa_binary,101,64,01121628_tsa-bin_NorBERT_3_large,/scratch/project_465000144/egilron/sq_label/01...,16,5e-05,NaN,0.000000,0.000000,0.000000,0.934185,1.0,135
16,01121628,tsa-bin,lumi,1,01121628_tsa-bin_NorBERT_3_base_13.json,ltg/norbert3-base,data/tsa_binary,101,64,01121628_tsa-bin_NorBERT_3_base,/scratch/project_465000144/egilron/sq_label/01...,16,1e-06,NaN,0.000000,0.000000,0.000000,0.934185,1.0,135
32,01121628,tsa-bin,lumi,13,01121628_tsa-bin_XLM-R_base_08.json,xlm-roberta-base,data/tsa_binary,101,64,01121628_tsa-bin_XLM-R_base,/scratch/project_465000144/egilron/sq_label/01...,16,1e-05,0.200411,0.470779,0.496009,0.483065,0.947232,13.0,1755
48,01121628,tsa-bin,lumi,13,01121628_tsa-bin_NB-Roberta_base_09.json,NbAiLab/nb-roberta-base-ncc-plus-scandi-1e4,data/tsa_binary,101,64,01121628_tsa-bin_NB-Roberta_base,/scratch/project_465000144/egilron/sq_label/01...,16,1e-05,0.198927,0.479915,0.517674,0.498080,0.948661,13.0,1755
64,01121628,tsa-bin,lumi,16,01121628_tsa-bin_NorBERT_3_small_06.json,ltg/norbert3-small,data/tsa_binary,101,64,01121628_tsa-bin_NorBERT_3_small,/scratch/project_465000144/egilron/sq_label/01...,16,1e-05,0.226876,0.336746,0.337514,0.337130,0.940902,16.0,2160
80,01121628,tsa-bin,lumi,1,01121628_tsa-bin_NorBERT_3_base_01.json,ltg/norbert3-base,data/tsa_binary,101,64,01121628_tsa-bin_NorBERT_3_base,/scratch/project_465000144/egilron/sq_label/01...,16,5e-05,NaN,0.000000,0.000000,0.000000,0.934185,1.0,135
96,01121628,tsa-bin,lumi,16,01121628_tsa-bin_XLM-R_base_02.json,xlm-roberta-base,data/tsa_binary,101,64,01121628_tsa-bin_XLM-R_base,/scratch/project_465000144/egilron/sq_label/01...,16,5e-05,0.355596,0.434739,0.493729,0.462360,0.941674,16.0,2160
112,01121628,tsa-bin,lumi,1,01121628_tsa-bin_NorBERT_3_base_07.json,ltg/norbert3-base,data/tsa_binary,101,64,01121628_tsa-bin_NorBERT_3_base,/scratch/project_465000144/egilron/sq_label/01...,16,1e-05,NaN,0.000000,0.000000,0.000000,0.934185,1.0,135
128,01121628,tsa-bin,lumi,16,01121628_tsa-bin_XLM-R_base_14.json,xlm-roberta-base,data/tsa_binary,101,64,01121628_tsa-bin_XLM-R_base,/scratch/project_465000144/egilron/sq_label/01...,16,1e-06,0.207057,0.351429,0.140251,0.200489,0.939319,16.0,2160
144,01121628,tsa-bin,lumi,16,01121628_tsa-bin_NB-Roberta_base_15.json,NbAiLab/nb-roberta-base-ncc-plus-scandi-1e4,data/tsa_binary,101,64,01121628_tsa-bin_NB-Roberta_base,/scratch/project_465000144/egilron/sq_label/01...,16,1e-06,0.195129,0.381513,0.258837,0.308424,0.940979,16.0,2160
